In [ ]:
import tkinter as tk
import threading
import time
import winsound
import pyttsx3

words = ["I", "am", "a", "student"]
word_to_idx = {"I": 0, "am": 1, "a": 2, "student": 3}
idx_to_word = {0: "I", 1: "am", 2: "a", 3: "student"}

input_words = ["I", "am", "a"]
target_word = "student"
input_indices = [word_to_idx[w] for w in input_words]
target_index = word_to_idx[target_word]

def one_hot(index, size):
    vec = [0] * size
    vec[index] = 1
    return vec

vocab_size = len(words)
hidden_size = 8

Wxh = [[(i+j)*0.01 for j in range(vocab_size)] for i in range(hidden_size)]
Whh = [[(i-j)*0.01 for j in range(hidden_size)] for i in range(hidden_size)]
Why = [[(i*j)*0.01 for j in range(hidden_size)] for i in range(vocab_size)]
bh = [0 for _ in range(hidden_size)]
by = [0 for _ in range(vocab_size)]

def tanh(x):
    return [(2/(1+pow(2.71828, -2*i)))-1 for i in x]

def tanh_derivative(x):
    return [1 - i*i for i in x]

def softmax(x):
    max_x = max(x)
    exps = [pow(2.71828, i - max_x) for i in x]
    sum_exps = sum(exps)
    return [j/sum_exps for j in exps]

def dot_product(mat, vec):
    result = []
    for row in mat:
        res = 0
        for i in range(len(vec)):
            res += row[i] * vec[i]
        result.append(res)
    return result

def add_vectors(v1, v2):
    return [v1[i]+v2[i] for i in range(len(v1))]

def forward(X):
    h = [0 for _ in range(hidden_size)]
    for x in X:
        h = tanh(add_vectors(dot_product(Wxh, x), add_vectors(dot_product(Whh, h), bh)))
    y_pred = softmax(add_vectors(dot_product(Why, h), by))
    return y_pred, h

def backward(X, y_true, y_pred, h, learning_rate=0.1):
    dy = [y_pred[i] - y_true[i] for i in range(len(y_true))]
    for i in range(len(Why)):
        for j in range(len(Why[0])):
            Why[i][j] -= learning_rate * dy[i] * h[j]
    for i in range(len(by)):
        by[i] -= learning_rate * dy[i]
    dh = [0]*hidden_size
    for i in range(hidden_size):
        for j in range(vocab_size):
            dh[i] += Why[j][i] * dy[j]
    dh_raw = [dh[i] * tanh_derivative(h)[i] for i in range(hidden_size)]
    for i in range(len(Wxh)):
        for j in range(len(Wxh[0])):
            Wxh[i][j] -= learning_rate * dh_raw[i] * X[-1][j]
    for i in range(len(Whh)):
        for j in range(len(Whh[0])):
            Whh[i][j] -= learning_rate * dh_raw[i] * h[j]
    for i in range(len(bh)):
        bh[i] -= learning_rate * dh_raw[i]

X = [one_hot(i, vocab_size) for i in input_indices]
y = one_hot(target_index, vocab_size)

for epoch in range(500):
    y_pred, h = forward(X)
    backward(X, y, y_pred, h)

def predict_word():
    original_color = predict_button.cget("bg")
    predict_button.config(bg="#00ccff")
    window.update()
    time.sleep(0.15)
    predict_button.config(bg=original_color)
    word1 = entry1.get()
    word2 = entry2.get()
    word3 = entry3.get()
    user_input = [word1, word2, word3]
    try:
        user_indices = [word_to_idx[w] for w in user_input]
    except KeyError:
        result_label.config(text="Unknown word entered!", fg="#ff3333")
        return
    X_user = [one_hot(i, vocab_size) for i in user_indices]
    pred_probs, _ = forward(X_user)
    pred_idx = pred_probs.index(max(pred_probs))
    predicted_word = idx_to_word[pred_idx]
    threading.Thread(target=lambda: winsound.Beep(500, 200)).start()
    result_label.config(text=f"Prediction: {predicted_word}", fg="#ff4d4d")
    window.update()
    time.sleep(0.2)
    result_label.config(fg="#ffd700")
    engine = pyttsx3.init()
    rate = rate_slider.get()
    engine.setProperty('rate', rate)
    voice_choice = voice_var.get()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[voice_choice].id)
    engine.say(predicted_word)
    engine.runAndWait()

window = tk.Tk()
window.title("RNN Text Predictor")
window.geometry("450x550")
window.configure(bg="#1f1f2e")

title_label = tk.Label(window, text="Predict the 4th Word", font=("Arial", 20, "bold"), bg="#1f1f2e", fg="#00ffd5")
title_label.pack(pady=10)

frame1 = tk.Frame(window, bg="#1f1f2e")
frame1.pack(pady=5)
label1 = tk.Label(frame1, text="First Word:", font=("Arial", 16), bg="#1f1f2e", fg="#ffffff")
label1.pack(side="left", padx=10)
entry1 = tk.Entry(frame1, font=("Arial", 16), bg="#33334d", fg="#ffffff", justify="center")
entry1.pack(side="left")

frame2 = tk.Frame(window, bg="#1f1f2e")
frame2.pack(pady=5)
label2 = tk.Label(frame2, text="Second Word:", font=("Arial", 16), bg="#1f1f2e", fg="#ffffff")
label2.pack(side="left", padx=10)
entry2 = tk.Entry(frame2, font=("Arial", 16), bg="#33334d", fg="#ffffff", justify="center")
entry2.pack(side="left")

frame3 = tk.Frame(window, bg="#1f1f2e")
frame3.pack(pady=5)
label3 = tk.Label(frame3, text="Third Word:", font=("Arial", 16), bg="#1f1f2e", fg="#ffffff")
label3.pack(side="left", padx=10)
entry3 = tk.Entry(frame3, font=("Arial", 16), bg="#33334d", fg="#ffffff", justify="center")
entry3.pack(side="left")

voice_var = tk.IntVar()
voice_var.set(0)
voice_choice_frame = tk.Frame(window, bg="#1f1f2e")
voice_choice_frame.pack(pady=5)
voice_label = tk.Label(voice_choice_frame, text="Select Voice:", bg="#1f1f2e", fg="#ffffff")
voice_label.pack(side="left", padx=10)
voice_male = tk.Radiobutton(voice_choice_frame, text="Male", variable=voice_var, value=0, bg="#1f1f2e", fg="#ffffff")
voice_male.pack(side="left", padx=10)
voice_female = tk.Radiobutton(voice_choice_frame, text="Female", variable=voice_var, value=1, bg="#1f1f2e", fg="#ffffff")
voice_female.pack(side="left", padx=10)

speed_label = tk.Label(window, text="Speech Speed:", bg="#1f1f2e", fg="#ffffff")
speed_label.pack(pady=10)
rate_slider = tk.Scale(window, from_=50, to=300, orient="horizontal", bg="#1f1f2e", fg="#ffffff", troughcolor="#ff0066", sliderlength=20)
rate_slider.set(150)
rate_slider.pack(pady=10)

predict_button = tk.Button(window, text="Predict", command=predict_word, font=("Arial", 16, "bold"), bg="#ff0066", fg="white", activebackground="#ff3385")
predict_button.pack(pady=20)

result_label = tk.Label(window, text="", font=("Arial", 18), bg="#1f1f2e", fg="#ffd700")
result_label.pack()

window.mainloop()
